# 데이터 전처리 부분 (고려 X)

In [5]:
import pandas as pd
from datetime import datetime, timedelta
from pandas.api.types import CategoricalDtype # 데이터프레임에서 범주형 데이터(categorical data)의 데이터 타입을 정의할 때 사용합니다.
pd.set_option('mode.chained_assignment',None) 

# 1. Covid19 원본 데이터셋 로딩
df_covid19 = pd.read_csv("owid-covid-data.csv")

# 2. 전체 데이터셋 중 최근 100일간의 데이터를 필터링한 df_covid19_100생성
df_covid19['date'] = pd.to_datetime(df_covid19['date'],format='%Y-%m-%d')

# 대륙 데이터와 최종 데이터로부터 100일 전 데이터 필터링
df_covid19_100 = df_covid19[(df_covid19['iso_code'].isin(['KOR',"OWID_ASI",'OWID_EUR',"OWID_OCE","OWID_NAM",'OWID_SAM','OWID_AFR'])) & (df_covid19['date']>=(max(df_covid19['date'] - timedelta(days=100))))]

# 대륙명을 한글로 변환
df_covid19_100.loc[df_covid19_100['location'] == 'South Korea',"location"] = '한국'
df_covid19_100.loc[df_covid19_100['location'] == 'Asia',"location"] = '아시아'
df_covid19_100.loc[df_covid19_100['location'] == 'Europe',"location"] = '유럽'
df_covid19_100.loc[df_covid19_100['location'] == 'Oceania',"location"] = '오세아니아'
df_covid19_100.loc[df_covid19_100['location'] == 'North America',"location"] = '북미'
df_covid19_100.loc[df_covid19_100['location'] == 'South America',"location"] = '남미'
df_covid19_100.loc[df_covid19_100['location'] == 'Africa',"location"] = '아프리카'

# 이산형 변수 설정
ord = CategoricalDtype(categories=['한국','아시아','유럽','북미','남미','아프리카','오세아니아'],ordered=True)

df_covid19_100['location'] = df_covid19_100['location'].astype(ord)

# date로 정리
df_covid19_100 = df_covid19_100.sort_values(by='date')

# 3. df_covid19_100을 한국과 각 대륙별 열로 배치한 넓은 형태의 데이터프레임 변환
df_covid19_100_wide = df_covid19_100.loc[:,['date','location','new_cases','people_fully_vaccinated_per_hundred']].rename(columns={'new_cases':'확진자','people_fully_vaccinated_per_hundred':'백신접종완료자'})

display(df_covid19_100_wide.head())

df_covid19_100_wide = df_covid19_100_wide.pivot(index='date',columns='location',values=['확진자','백신접종완료자']).sort_values(by='date')
display(df_covid19_100_wide.head())

df_covid19_100_wide.columns = ['확진자_한국','확진자_아시아','확진자_유럽','확진자_북미','확진자_남미','확진자_아프리카','확진자_오세아니아','백신접종완료자_한국',
                               '백신접종완료자_아시아','백신접종완료자_유럽','백신접종완료자_북미','백신접종완료자_남미','백신접종완료자_아프리카','백신접종완료자_오세아니아']

# 4. covid19 데이터를 국가별로 요약한 df_covid19_stat 생성
df_covid19_stat = df_covid19.groupby(['iso_code','continent','location'],dropna=True).agg(
    인구수 = ('population','max'),
    전체사망자수 = ('new_deaths','sum'),
    백신접종자완료자수 = ('people_fully_vaccinated','max'),
    인구백명당백신접종완료율 = ('people_fully_vaccinated_per_hundred','max'),    
    인구백명당부스터접종자수 = ('total_boosters_per_hundred','max')).reset_index()

df_covid19_stat['십만명당사망자수'] = round(df_covid19_stat['전체사망자수'] / df_covid19_stat['인구수'] * 10000,5)

df_covid19_stat['백신접종완료율'] = df_covid19_stat['백신접종자완료자수'] / df_covid19_stat['인구수'] 

## 여백 설정을 위한 변수 설정
margins_P = {'t':50,'b':25,'l':25,'r':25}

,date,location,확진자,백신접종완료자
2016,2022-10-03,아프리카,754.0,22.76
163005,2022-10-03,북미,45451.0,64.43
207753,2022-10-03,남미,15450.0,76.48
12551,2022-10-03,아시아,84967.0,71.77
208839,2022-10-03,한국,16423.0,85.73


확진자                                                         \
location         한국       아시아        유럽       북미       남미    아프리카    오세아니아   
date                                                                         
2022-10-03  16423.0   84967.0  168183.0  45451.0  15450.0   754.0   5145.0   
2022-10-04  34710.0  149537.0  384493.0  61078.0   2379.0  1217.0   1536.0   
2022-10-05  28603.0  149638.0  330710.0  80045.0  18412.0   780.0   3098.0   
2022-10-06  22259.0  127116.0  394838.0  71088.0  14690.0   904.0  31919.0   
2022-10-07  19379.0  124355.0  375601.0  67185.0   4412.0   533.0   2517.0   

           백신접종완료자                                            
location        한국    아시아     유럽     북미     남미   아프리카  오세아니아  
date                                                          
2022-10-03   85.73  71.77  66.58  64.43  76.48  22.76  61.67  
2022-10-04   85.73  71.78  66.62  64.44  76.48  22.76  61.67  
2022-10-05   85.73  71.80  66.63  64.45  76.48  22.76  61.68  
2022-10-06   85.73  71.80  66.63  64.46  76.48  22.76  61.68  
2022-10-07   85.73  71.81  66.63  64.79  76.53  22.76  61.68

In [6]:
import pandas as pd

df_취업률 = pd.read_excel('2021년 학과별 고등교육기관 취업통계.xlsx',
                          sheet_name = '학과별',
                          skiprows=(13),
                          header=0)
df_취업률 = pd.concat([df_취업률.iloc[:,0:8],
                      df_취업률.loc[:,df_취업률.columns.str.endswith('계')],
                      df_취업률.loc[:,'입대자']],axis=1)
df_취업률_500 = df_취업률.loc[(df_취업률['졸업자_계'] < 500)]
df_취업률_500 = df_취업률_500.iloc[range(0,len(df_취업률_500.index),4)]
df_취업률_500 = df_취업률_500.rename(columns={'졸업자_계':'졸업자수',
                                        '취업률_계':'취업률','취업자_합계_계':'취업자수'})
df_취업률_500

,조사기준일,학제,과정구분,대계열,중계열,소계열,학과코드,학과명,졸업자수,취업률,...,1차 유지취업자_계,1차 유지취업률_계,2차 유지취업자_계,2차 유지취업률_계,3차 유지취업자_계,3차 유지취업률_계,4차 유지취업자_계,4차 유지취업률_계,입학당시 기취업자_계,입대자
0,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,일본어,C01010100002,관광일본어과,79,29.6,...,10,76.9,8,61.5,7,53.8,5,38.5,0,14
4,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,일본어,C01010100013,일본어과,63,46.2,...,14,70.0,13,65.0,8,40.0,8,40.0,4,4
8,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,일본어,C01010100049,항공호텔관광학부 일본어통역전공,40,37.5,...,5,62.5,5,62.5,5,62.5,5,62.5,0,2
12,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,중국어,C01010200003,관광중국어과,225,51.9,...,68,84.0,62,76.5,53,65.4,52,64.2,6,9
16,2021.12.31,전문대학,전문대학과정,인문계열,언어ㆍ문학,중국어,C01010200011,중국어과,5,60.0,...,3,100.0,2,66.7,2,66.7,2,66.7,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9388,2021.12.31,기능대학,전문대학과정,자연계열,생물ㆍ화학ㆍ환경,생물,C05020100066,바이오품질관리과,28,89.3,...,25,100.0,25,100.0,23,92.0,23,92.0,0,0
9392,2021.12.31,기능대학,전문대학과정,의약계열,치료ㆍ보건,의료장비,C06020300002,의료공학과,1,0.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0
9396,2021.12.31,기능대학,전문대학과정,예체능계열,디자인,패션디자인,C07010300011,패션디자인과,21,87.5,...,8,80.0,6,60.0,5,50.0,4,40.0,0,2
9400,2021.12.31,기능대학,전문대학과정,예체능계열,디자인,기타디자인,C07010400214,금형디자인과,326,80.3,...,209,89.7,197,84.5,184,79.0,178,76.4,15,22


# Data 속성 (go.Figure()에서 할당하기)

In [7]:
import plotly.graph_objects as go # go.Figure()의 data 매개변수인 data 속성들로 구성된 딕셔너리를 할당
go.Figure(
    data = [{
        'type':'scatter','mode':'markers', # scatter 트레이스의 markers 속성 설정
        ## X,y 축에 변수 매핑
        'x' : df_covid19_100.loc[df_covid19_100['iso_code'] == 'KOR','date'],
        'y' : df_covid19_100.loc[df_covid19_100['iso_code'] == 'KOR','new_cases'],
        ## 마커 색상 설정
        'marker' : {'color':'#264E86'}},
        ## scatter 트레이스의 lines 속성 설정
        {'type':'scatter','mode':'lines',
        'x' : df_covid19_100.loc[df_covid19_100['iso_code'] == 'KOR','date'],
        'y' : df_covid19_100.loc[df_covid19_100['iso_code'] == 'KOR','new_cases'],
        'line': {'color':"#5E88FC",'dash':'dash'}} # dash는 점선 표시이다.
    ]
)

# Layout 속성 (go.Figure()에서 할당)

In [8]:
fig = go.Figure(
    data = {
        'type' : 'scatter',
        'mode' : 'markers+lines',
        'x' : df_covid19_100.loc[df_covid19_100['iso_code']=='KOR','date'],
        'y' : df_covid19_100.loc[df_covid19_100['iso_code']=='KOR','new_cases'],
        'marker' : {'color':'#264E86'},
        'line' : {'color':'#5E88FC','dash':'dash'}
    },
    layout = { ## layout 속성의 설정
        'title':'코로나 19 발생현황', # 전체 제목 설정
        'xaxis':{'title':'날짜','showgrid':False}, # X축 layout 속성 설정
        'yaxis':{'title':'확진자수'}, # y축 layout 속성 설정
        'margin': margins_P # 여백 설정
    })
fig.show()

# 입력 데이터 구조 확인

In [9]:
print(fig)

Figure({
    'data': [{'line': {'color': '#5E88FC', 'dash': 'dash'},
              'marker': {'color': '#264E86'},
              'mode': 'markers+lines',
              'type': 'scatter',
              'x': array([datetime.datetime(2022, 10, 3, 0, 0),
                          datetime.datetime(2022, 10, 4, 0, 0),
                          datetime.datetime(2022, 10, 5, 0, 0),
                          datetime.datetime(2022, 10, 6, 0, 0),
                          datetime.datetime(2022, 10, 7, 0, 0),
                          datetime.datetime(2022, 10, 8, 0, 0),
                          datetime.datetime(2022, 10, 9, 0, 0),
                          datetime.datetime(2022, 10, 10, 0, 0),
                          datetime.datetime(2022, 10, 11, 0, 0),
                          datetime.datetime(2022, 10, 12, 0, 0),
                          datetime.datetime(2022, 10, 13, 0, 0),
                          datetime.datetime(2022, 10, 14, 0, 0),
                          datetime.datet

- plotly 객체는 data,layout,frame 속성값들을 사용자 설정하여 JSON 파일의 구조로 데이터 형식들을 만드는 것이다.

- data 속성은 트레이스로 구성되어 데이터를 시각화하는 것이 가장 핵심적 요소가 된다.


# add_trace로 data속성 부여 1번째 방법
- go.Figure()로 속성을 부여할 수 있지만 너무 길고 오류 발생 확률이 크다.
- 그래서 add_trace로 trace에 대한 속성들을 부여한다.

In [14]:
fig = go.Figure() # 초기화

fig.add_trace({ ## 트레이스 추가
    'type':'scatter',
    'mode':'markers',
    'x': df_취업률_500['졸업자수'], 'y':df_취업률_500['취업자수'],
    'marker': {'size':5,'color':'darkblue'} ## 마커 크기와 색상 설정
})
fig.show()


# data 속성 추가 방법 2
- plotly.graph_objects에서 제공하는 각각의 트레이스 함수를 사용 ex) go.Scatter()

- go.Scatter() {}형태로 속성명과 속성값 할당

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        {
            'mode':'markers',
            'x':df_취업률_500['졸업자수'],'y':df_취업률_500['취업자수'],
            'marker':{'size':3,'color':'darkblue'}
        }
    )
)
fig.show()

- 파라미터로 할당할 때

In [16]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(mode='markers',x=df_취업률_500['졸업자수'],y=df_취업률_500['취업자수'],
               marker=dict(size=3,color='darkblue'))
             )

# 트레이스 공통 속성 
- 40개가 넘는 트레이스 제공하지만 파라미터들이 다 다르지만 공통된 속성이 있다.

In [23]:
fig = go.Figure()

for 대계열, group in df_취업률_500.groupby('대계열'):
    fig.add_trace({
        'type':'scatter','mode':'markers',
        'x':group['졸업자수'],'y':group['취업자수'],
        'name':대계열
    })
fig.show()

# Opacity (투명도)

In [37]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        {
            'mode':'markers',
            'x':df_취업률_500['졸업자수'],'y':df_취업률_500['취업자수'],
            'marker':{'size':5,'color':'darkblue'},
            'opacity':0.4
        }
    )
)
fig.update_layout(title = {'text':'Opacity를 외부에 설정'})
fig.show()

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        {   
            'mode':'markers',
            'x':df_취업률_500['졸업자수'],'y':df_취업률_500['취업자수'],
            'marker':{'size':3,'color':'darkblue','opacity':0.4}
        }
    )
)
fig.update_layout(title = {'text':'Opacity를 내부에 설정'})
fig.show()

# ShowLegend

In [40]:
fig = go.Figure()

for 대계열,group in df_취업률_500.groupby('대계열'):
    fig.add_trace(
        {'type':'scatter',
         'mode':'markers',
         'x':group['졸업자수'],
         'y':group['취업자수'],
         'name':대계열,
         'showlegend':False} ## 범례 표시를 하지 않는다.
    )
fig.show()